In [1]:
# Import necessary libraries
import cv2
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
import cv2
import numpy as np
import time
import csv
import os
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30)

class CustomerTracker:
    def __init__(self, index, store_layout_zones=None, dwell_time_threshold=60):
        self.tracks = defaultdict(list)
        self.dwell_times = defaultdict(float)
        self.heatmap_data = np.zeros((720, 1280))
        self.store_zones = store_layout_zones or {
            'entrance': [(0, 0, 400, 720)],
            'checkout': [(880, 0, 1280, 720)],
            'aisle': [(400, 0, 880, 720)],
            'exit': [(0, 600, 1280, 720)]
        }
        self.zone_visits = defaultdict(int)
        self.last_positions = {}
        self.start_times = {}
        self.dwell_time_threshold = dwell_time_threshold
        self.last_alert_time = {}
        self.video_index = index
        self.alerts = []

    def update_tracking(self, person_id, x, y, w, h):
        center_x, center_y = x + w // 2, y + h // 2
        self.tracks[person_id].append((center_x, center_y))
        self.heatmap_data[max(0, y):min(720, y + h), max(0, x):min(1280, x + w)] += 1

        current_zone = self.get_current_zone(center_x, center_y)
        if person_id not in self.last_positions:
            self.last_positions[person_id] = current_zone
            self.start_times[person_id] = time.time()
        else:
            if self.last_positions[person_id] != current_zone:
                dwell_time = time.time() - self.start_times[person_id]
                self.dwell_times[self.last_positions[person_id]] += dwell_time
                self.start_times[person_id] = time.time()
                self.zone_visits[current_zone] += 1

            if current_zone == "exit":
                self.alerts.append((person_id, "Exited the store"))

            if self.dwell_times.get(self.last_positions[person_id], 0) > self.dwell_time_threshold:
                self.trigger_alert(person_id, self.last_positions[person_id], self.dwell_times[self.last_positions[person_id]])

            self.last_positions[person_id] = current_zone

    def trigger_alert(self, person_id, zone, dwell_time):
        current_time = time.time()
        if person_id in self.last_alert_time and (current_time - self.last_alert_time[person_id] < 30):
            return
        self.last_alert_time[person_id] = current_time
        alert_msg = f"ALERT: Customer {person_id} in {zone} > {dwell_time:.1f}s"
        self.alerts.append((person_id, alert_msg))

    def get_current_zone(self, x, y):
        for zone_name, boxes in self.store_zones.items():
            for (x1, y1, x2, y2) in boxes:
                if x1 <= x <= x2 and y1 <= y <= y2:
                    return zone_name
        return 'other'

    def export_csv(self):
        filename = f"movement_data_{self.video_index + 1}.csv"
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Person ID", "X", "Y"])
            for pid, positions in self.tracks.items():
                for x, y in positions:
                    writer.writerow([pid, x, y])

    def draw_zones(self, frame):
        for zone_name, boxes in self.store_zones.items():
            for (x1, y1, x2, y2) in boxes:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, zone_name, (x1 + 5, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    def draw_alerts(self, frame):
        for i, (_, msg) in enumerate(self.alerts[-5:]):
            cv2.putText(frame, msg, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    def generate_heatmap(self):
        return self.heatmap_data


def analyze_video(video_path, index, all_heatmaps):
    print(f"\n--- Processing video {index + 1}: {video_path} ---")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file. Skipping...")
        return

    net = cv2.dnn.readNet("yolov4-tiny.weights", "yolov4-tiny.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    tracker_obj = CustomerTracker(index)
    width, height = int(cap.get(3)), int(cap.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(f"output_video_{index + 1}.mp4", fourcc, 30.0, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(net.getUnconnectedOutLayersNames())

        boxes, confidences, class_ids = [], [], []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.3 and classes[class_id] == "person":
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.4)
        detections = []

        if indexes is not None and len(indexes) > 0:
            for i in indexes.flatten():
                if 0 <= i < len(boxes):
                    x, y, w, h = boxes[i]
                    detections.append(([x, y, x + w, y + h], confidences[i], class_ids[i]))

        tracked_objects = tracker.update_tracks(detections, frame=frame)
        for track in tracked_objects:
            if not track.is_confirmed():
                continue
            track_id, bbox = track.track_id, track.to_tlbr()
            x1, y1, x2, y2 = map(int, bbox)
            tracker_obj.update_tracking(track_id, x1, y1, x2 - x1, y2 - y1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        tracker_obj.draw_zones(frame)
        tracker_obj.draw_alerts(frame)
        output_video.write(frame)

    cap.release()
    output_video.release()
    cv2.destroyAllWindows()

    tracker_obj.export_csv()
    heatmap = tracker_obj.generate_heatmap()
    all_heatmaps.append(heatmap)
    plt.figure(figsize=(12, 8))
    sns.heatmap(heatmap, cmap='YlOrRd')
    plt.title(f'Customer Heatmap - Video {index + 1}')
    plt.savefig(f'customer_heatmap_{index + 1}.png')
    plt.close()


def combine_heatmaps(heatmaps):
    combined = np.sum(heatmaps, axis=0)
    plt.figure(figsize=(12, 8))
    sns.heatmap(combined, cmap='YlOrRd')
    plt.title('Combined Customer Movement Heatmap')
    plt.savefig('combined_heatmap.png')
    plt.close()


# Run analysis on multiple videos
video_files = [
    "video/HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCamerasnet retail store.mp4",
    "video/HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCamerasnet retail store-3-23.mp4",
    "video/new_codec_HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCamerasnet retail store-3-23.mp4"
]

all_heatmaps = []
for idx, path in enumerate(video_files):
    analyze_video(path, idx, all_heatmaps)

combine_heatmaps(all_heatmaps)



--- Processing video 1: video/HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCamerasnet retail store.mp4 ---

--- Processing video 2: video/HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCamerasnet retail store-3-23.mp4 ---

--- Processing video 3: video/new_codec_HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCamerasnet retail store-3-23.mp4 ---
